In [1]:
%matplotlib inline
import os
import importlib

import xarray as xr
import numpy as np

import project as P
import data_catalog

import esmlab

In [2]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster
import dask

USER = os.environ['USER']

Nnodes = 4
processes = 18
project = 'NCGD0011'

cluster = PBSCluster(queue='regular',
                     cores = 18,
                     processes = processes,
                     memory = '100GB',          
                     project = project,
                     walltime = '04:00:00',
                     local_directory=f'/glade/scratch/{USER}/dask-tmp')
client = Client(cluster)
cluster.scale(processes*Nnodes)

Here is a summary of the things I need from you per our discussion this morning:
1. Annual mean files of the sigma_coord variables
2. Subset files of the North Pacific for the sigma_coord annmean variables
**As soon as I can get back on the system I will send over the zero based indexing for the region of interest, nlat[start:end] nlong[start:end]

3. The annmean, monclim, monanom files from the new case that includes 13C
case = 'g.e21a01d.G1850ECOIAF.T62_g17.extraterr-fe.001'
varlist = ['DIC', 'DI13C', 'CISO_DIC_d13C', 'ALK', 'TEMP', 'HMXL', 'SALT', 'Jint_100m_DIC', 'RHO', 'NO3', 'PO4']
**I think the climatologies should be over the years 1983-2015... (to best match up with obs records of 1983-2017 and 1989-2016?)

Compute annual means of sigma-coord files

In [4]:
data_catalog.set_catalog('cesm_dple')
importlib.reload(P)

recipes = 'analysis_recipes.yml'
experiment = 'g.e11_LENS.GECOIAF.T62_g16.009_sigma_coord'

for v in data_catalog.find_in_index(experiment=experiment).variable.unique():
    co = P.process_data_source(analysis_name='annmean',
                               analysis_recipes=recipes,
                               experiment=experiment,
                               variable=v, 
                               clobber=False)

active catalog: cesm_dple


INFO:root:Applying operator: <function compute_ann_mean at 0x2aab67582840>
INFO:root:writing /glade/scratch/mclong/calcs/processed_collections/cesm_dple.g.e11_LENS.GECOIAF.T62_g16.009_sigma_coord.000.annmean.NO3.nc


time_bound


INFO:root:Applying operator: <function compute_ann_mean at 0x2aab67582840>
INFO:root:writing /glade/scratch/mclong/calcs/processed_collections/cesm_dple.g.e11_LENS.GECOIAF.T62_g16.009_sigma_coord.000.annmean.O2.nc


time_bound


INFO:root:Applying operator: <function compute_ann_mean at 0x2aab67582840>
INFO:root:writing /glade/scratch/mclong/calcs/processed_collections/cesm_dple.g.e11_LENS.GECOIAF.T62_g16.009_sigma_coord.000.annmean.SALT.nc


time_bound


INFO:root:Applying operator: <function compute_ann_mean at 0x2aab67582840>
INFO:root:writing /glade/scratch/mclong/calcs/processed_collections/cesm_dple.g.e11_LENS.GECOIAF.T62_g16.009_sigma_coord.000.annmean.TEMP.nc


time_bound


In [ ]:
importlib.reload(P)

data_catalog.set_catalog('cesm_dple')

recipes = 'analysis_recipes.yml'
experiment = 'g.e11_LENS.GECOIAF.T62_g16.009'

varlist = ['Jint_100m_ALK', 'Jint_100m_DIC', 'Jint_100m_DOC', 
           'Jint_100m_Fe', 'Jint_100m_NH4', 'Jint_100m_NO3', 
           'Jint_100m_O2', 'Jint_100m_PO4', 'Jint_100m_SiO3', 
           'DIC', 'PO4', 'O2', 'TEMP', 'SALT', 'NO3', 'Fe', 
           'SSH', 'PD', 'NOx_FLUX', 'FG_CO2']
varlist = varlist+[f'tend_zint_100m_{v}' for v in ['DIC', 'NO3']]

for v in varlist:
    co = P.process_data_source(experiment=experiment,
                               analysis_name='annmean',
                               analysis_recipes=recipes,
                               variable=v, 
                               clobber=False)
    
    co = P.process_data_source(experiment=experiment,
                           analysis_name='monclim_1983-2015',                               
                           analysis_recipes=recipes,
                           variable=v, 
                           clobber=False)  

    co = P.process_data_source(experiment=experiment,
                           analysis_name='monanom_rel-1983-2015',                               
                           analysis_recipes=recipes,
                           variable=v, 
                           clobber=False)       

active catalog: cesm_dple


In [4]:
data_catalog.set_catalog('cesm2_runs')

experiment = 'g.e21a01d.G1850ECOIAF.T62_g17.extraterr-fe.001'

#'DI13C', 'CISO_DIC_d13C', 
for v in ['FG_CO2', 'DIC', 'ALK', 'TEMP', 'HMXL', 'SALT', 'Jint_100m_DIC', 'RHO', 'NO3', 'PO4']:
    co = P.process_data_source(experiment=experiment,
                               analysis_name='annmean',
                               analysis_recipes=recipes,
                               variable=v, 
                               clobber=False)
    
    co = P.process_data_source(experiment=experiment,
                           analysis_name='monclim_1983-2009',                               
                           analysis_recipes=recipes,
                           variable=v, 
                           clobber=False)   
    

active catalog: cesm2_runs


FileNotFoundError: [Errno 2] No such file or directory: b'/glade/scratch/mclong/archive/g.e21a01d.G1850ECOIAF.T62_g17.extraterr-fe.001/ocn/proc/tseries/month_1/g.e21a01d.G1850ECOIAF.T62_g17.extraterr-fe.001.pop.h.FG_CO2.024101-031012.nc'

In [ ]:
importlib.reload(P)

co = P.process_data_source(analysis_name='monanom',
                               experiment='ctrl_ocean-ice-core',
                               variable='MLD_03',
                               isderived=True,
                               clobber=False)

INFO:root:Applying operator: <function derive_var_MLD at 0x2aacb6768158>


In [64]:
data_catalog.get_entries(experiment='hindcast_sigma_coord', variable=['SALT','TEMP'])

{'case': ['g.e11_LENS.GECOIAF.T62_g16.009', 'g.e11_LENS.GECOIAF.T62_g16.009'],
 'component': ['ocn', 'ocn'],
 'date_range': ["['024901', '031612']", "['024901', '031612']"],
 'ensemble': [0, 0],
 'experiment': ['hindcast_sigma_coord', 'hindcast_sigma_coord'],
 'file_basename': ['g.e11_LENS.GECOIAF.T62_g16.009.pop.h.sigma.SALT.024901-031612.nc',
  'g.e11_LENS.GECOIAF.T62_g16.009.pop.h.sigma.TEMP.024901-031612.nc'],
 'files': ['/glade/p/cgd/oce/projects/DPLE_O2/sigma_coord/CESM-DPLE_POPCICEhindcast/g.e11_LENS.GECOIAF.T62_g16.009.pop.h.sigma.SALT.024901-031612.nc',
  '/glade/p/cgd/oce/projects/DPLE_O2/sigma_coord/CESM-DPLE_POPCICEhindcast/g.e11_LENS.GECOIAF.T62_g16.009.pop.h.sigma.TEMP.024901-031612.nc'],
 'grid': ['POP_gx1v6', 'POP_gx1v6'],
 'sequence_order': [0, 0],
 'stream': ['pop.h.sigma', 'pop.h.sigma'],
 'variable': ['SALT', 'TEMP'],
 'year_offset': [1699, 1699],
 'ctrl_branch_year': [nan, nan],
 'has_ocean_bgc': [nan, nan]}